<a href="https://colab.research.google.com/github/shivangsingh26/FL-BC-BTP/blob/master/fvProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow federated
!pip install gdown

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
import zipfile

# Unzip the datasets
zip_path = '/content/drive/My Drive/BTP.zip'
unzip_path = '/content/datasets'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_path)


Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_data(client_path):
    train_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        os.path.join(client_path, 'train'),
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        os.path.join(client_path, 'test'),
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    return train_generator, test_generator

clients = ['/content/datasets/Non-iid datasets/non_iid_subset_1','/content/datasets/Non-iid datasets/non_iid_subset_2','/content/datasets/Non-iid datasets/non_iid_subset_3','/content/datasets/Non-iid datasets/non_iid_subset_4',]
data_paths = [os.path.join(unzip_path, client) for client in clients]

train_generators = []
test_generators = []
for path in data_paths:
    train_gen, test_gen = load_data(path)
    train_generators.append(train_gen)
    test_generators.append(test_gen)

Found 19012 images belonging to 19 classes.
Found 4748 images belonging to 19 classes.
Found 19185 images belonging to 19 classes.
Found 4791 images belonging to 19 classes.
Found 19703 images belonging to 19 classes.
Found 4921 images belonging to 19 classes.
Found 19530 images belonging to 19 classes.
Found 4878 images belonging to 19 classes.


In [3]:
def create_model(num_classes=19):
    base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                                   include_top=False,
                                                   weights='imagenet')
    base_model.trainable = False

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


In [4]:
import numpy as np

# Define number of communication rounds and RR ratio
num_rounds = 15
RR = 0.5  # Example value for RR

# Initialize models for each client
client_models = [create_model(num_classes=19) for _ in range(4)]

# Function to perform the FedWPR aggregation
def fed_wpr(client_models, RR):
    new_weights_list = []

    # Get the weights from each client model
    client_weights = [model.get_weights() for model in client_models]

    # Initialize new weights for each client model
    for client_id in range(len(client_models)):
        new_weights = []
        for layer_weights in zip(*client_weights):
            weighted_sum = np.sum([w * RR for w in layer_weights], axis=0)
            personalized_weights = (1 - RR) * layer_weights[client_id] + weighted_sum
            new_weights.append(personalized_weights)
        new_weights_list.append(new_weights)

    return new_weights_list

for round_num in range(num_rounds):
    print(f'Round {round_num+1}/{num_rounds}')

    # Train each client model on its local data
    for i in range(4):
        print(f'Training client {i+1}')
        client_models[i].fit(train_generators[i], epochs=1, validation_data=test_generators[i])

    # Perform FedWPR aggregation
    new_weights_list = fed_wpr(client_models, RR)

    # Update each client model with its personalized weights
    for i in range(4):
        client_models[i].set_weights(new_weights_list[i])


9406464/9406464 [==============================] - 0s 0us/step
Round 1/15
Training client 1
595/595 [==============================] - 97s 152ms/step - loss: 0.7224 - accuracy: 0.7839 - val_loss: 0.2592 - val_accuracy: 0.9265
Training client 2
600/600 [==============================] - 96s 153ms/step - loss: 0.5435 - accuracy: 0.8453 - val_loss: 0.1467 - val_accuracy: 0.9616
Training client 3
616/616 [==============================] - 95s 148ms/step - loss: 0.6839 - accuracy: 0.8011 - val_loss: 0.2306 - val_accuracy: 0.9329
Training client 4
611/611 [==============================] - 98s 154ms/step - loss: 0.6558 - accuracy: 0.8086 - val_loss: 0.2275 - val_accuracy: 0.9330
Round 2/15
Training client 1
595/595 [==============================] - 88s 147ms/step - loss: 2.9855 - accuracy: 0.0537 - val_loss: 2.9378 - val_accuracy: 0.0545
Training client 2
600/600 [==============================] - 86s 143ms/step - loss: 2.9881 - accuracy: 0.0550 - val_loss: 2.9383 - val_accuracy: 0.0536
Tra

In [5]:
for i in range(4):
    loss, accuracy = client_models[i].evaluate(test_generators[i])
    print(f'Client {i+1} - Loss: {loss}, Accuracy: {accuracy}')

149/149 [==============================] - 18s 120ms/step - loss: 594.4900 - accuracy: 0.0545
Client 1 - Loss: 594.489990234375, Accuracy: 0.054549284279346466
150/150 [==============================] - 17s 115ms/step - loss: 1757352.0000 - accuracy: 0.0539
Client 2 - Loss: 1757352.0, Accuracy: 0.05385097116231918
154/154 [==============================] - 21s 138ms/step - loss: 1144.0861 - accuracy: 0.0526
Client 3 - Loss: 1144.0860595703125, Accuracy: 0.05263157933950424
153/153 [==============================] - 10s 66ms/step - loss: 1153.7388 - accuracy: 0.0531
Client 4 - Loss: 1153.73876953125, Accuracy: 0.05309553071856499
